In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import math

In [2]:
def FIFA(x):

    
    #Remove stars from the following columns
    
    x['SM']=x['SM'].str.replace('★','') 
    x['W/F']=x['W/F'].str.replace('★','')
    x['IR']=x['IR'].str.replace('★','')
    
  
    #change titles to lower case 
    lower = []
    for i in x.columns:
        lower.append(i.lower())
    x.columns = lower
    
    #Rename unclear column names
    x = x.rename(columns={'weight':'weight kg','loan date end':'on_loan', 'value':'value_€','wage':'wage_€','release clause':'release_clause_€','bp' : 'best position','a/w' : 'attacking workrate','d/w':'defensive workrate','ir' : 'international reputation','w/f':'weak foot','sm' : 'skill move','ova':'ovr'})
    
    
    #Puts underscore instead of spaces
    under_ = []
    for i in x.columns:
        if ' ' in i:
            under_.append(i.replace(' ', '_'))
        else:
            under_.append(i)
    x.columns = under_   

    
    #cleaned up the final columns (taking + out)
    def split(y):
        y = y.str.split("+", n = 1, expand = True)[0]
        return y 
    
    cols = x[['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']]
    for i in cols:
        x[i] = split(cols[i])
        x[i] = x[i].astype(int)

        
    #Drops unnecessary columns    
    coldrp = ['id','position','joined','team_&_contract','contract','crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys','dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control','acceleration', 'sprint_speed', 'agility', 'reactions' , 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure','marking', 'standing_tackle', 'sliding_tackle','gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

    x[i] = x.drop(coldrp, axis = 1, inplace = True)
    

    x['on_loan'] = x['on_loan'].fillna(0)
    
    numb = []
    for i in x['on_loan']:
        if i != 0:
            numb.append(1)
        else:
            numb.append(0)

    x['on_loan'] = numb
    
    def moneycon(x):

        money = []
        for i in x:
            i = i.replace("€",'')
            if 'K' in i:
                i = i.replace("K",'')
                i = i.replace(".",'')
                i = float(i)
                i = float(i * 1000)
                i = int(i)
                money.append(i)
            elif 'M' in i:
                i = i.replace("M",'')
                i = float(i)
                i = float(i * 1000000)
                i = int(i)
                money.append(i)
            else:
                money.append(i)   
            
        return money
    
    x['value_€'] = moneycon(x['value_€'])
    x['wage_€'] = moneycon(x['wage_€'])
    x['release_clause_€'] = moneycon(x['release_clause_€'])
    x['hits'] = moneycon(x['hits'])
    
    x['value_€'] = pd.to_numeric(x['value_€'])
    x['wage_€'] = pd.to_numeric(x['wage_€'])
    x['release_clause_€'] = pd.to_numeric(x['release_clause_€'])
    x['hits'] = pd.to_numeric(x['hits'])
    
    x['defensive_workrate'] = x['defensive_workrate'].fillna("U")
    x['defensive_workrate'] = x['defensive_workrate'].map({'Low':1,'Medium':2,'High':3, "U":0})
    x['attacking_workrate'] = x['attacking_workrate'].fillna("U")
    x['attacking_workrate'] = x['attacking_workrate'].map({'Low':1,'Medium':2,'High':3, "U":0})
    x['foot'] = x['foot'].fillna("U")
    x['foot'] = x['foot'].map({'Right':0,'Left':1})
    #x['gk'] = x['gk'].map({'None':0})
    
   # convint = x[['skill_move', 'weak_foot','international_reputation', 'hits']]
   # convint = convint.fillna(0)
   # for i in convint:
   #         x[i] = x[i].astype(int)
      
    x['international_reputation'] = x['international_reputation'].astype(int)
    #x['hits'] = x['hits'].astype(int)
    x['skill_move'] = x['skill_move'].astype(int)
    x['weak_foot'] = x['weak_foot'].astype(int)

    weight=[]
    for i in x['weight_kg']:
        i = i.replace('lbs','')
        i = float(i)*0.453592
        weight.append(round(i,2))
                     
    x['weight_kg'] = weight
    
    x = x[x['club'].isna()==False]
    
    
    x['heightm'] = x['height'].str.split("'", n = 1, expand = True)[0]
    x['heightcm'] = x['height'].str.split("'", n = 0, expand = True)[0]

    changetype = x[[]]

    heightmcon = []
    for i in x['heightm']:
        i = float(i) * 30.48
        i = int(i)
        heightmcon.append(i)
    x['heightm'] = heightmcon

    heightcmcon= []
    for i in x['heightcm']:
        i = float(i) * 2.54
        i = int(i)
        heightcmcon.append(i)
    x['heightcm'] = heightcmcon

    x['height'] = x['heightm']+x['heightcm']
    
    hdrop = x[['heightm','heightcm']]
    x[i] = x.drop(hdrop, axis = 1, inplace = True)
    
    x = x[['name','age','nationality','club','best_position','height','weight_kg','foot','growth','value_€','wage_€','release_clause_€','on_loan','attacking','skill','movement','power','mentality','defending','goalkeeping','total_stats','base_stats','weak_foot','skill_move','attacking_workrate','defensive_workrate','international_reputation','pac','sho','pas','dri','def','phy','hits','ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','ovr']]
    
    return x

In [3]:
fifa21 = pd.read_csv('/Users/hollydalton/Downloads/fifa21_train.csv')

In [4]:
fifa21 = FIFA(fifa21)

In [5]:
pd.set_option('display.max_columns', 500)
#fifa21.dtypes[5:]

In [6]:
fifa21

,name,age,nationality,club,best_position,height,weight_kg,foot,growth,value_€,wage_€,release_clause_€,on_loan,attacking,skill,movement,power,mentality,defending,goalkeeping,total_stats,base_stats,weak_foot,skill_move,attacking_workrate,defensive_workrate,international_reputation,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,ovr
0,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,164,73.03,0,1,525000,4000,801000,0,258,286,346,306,290,148,48,1682,357,4,2,3,2,1,69,51,63,63,51,60,3,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,64
1,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,197,72.12,0,0,8500000,23000,0,1,365,375,404,372,313,77,55,1961,412,3,4,3,1,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,77
2,S. Giovinco,33,Italy,Al Hilal,CAM,164,60.78,0,0,9000000,49000,15300000,0,336,424,424,308,332,80,21,1925,404,4,4,3,2,2,80,77,78,86,27,56,73,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,80
3,J. Evans,22,Wales,Swansea City,CDM,164,68.95,0,13,275000,4000,694000,0,242,259,282,277,257,168,42,1527,329,2,2,2,2,1,57,44,54,57,57,60,7,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,59
4,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,164,68.04,0,8,725000,2000,1400000,0,249,280,324,280,294,185,52,1664,360,2,3,1,2,1,66,44,60,64,60,66,4,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,197,76.20,1,5,625000,3000,891000,0,224,248,333,273,266,184,61,1589,347,3,2,2,2,1,75,28,56,59,60,69,3,48,48,48,55,52,52,52,55,54,54,54,58,56,56,56,58,63,60,60,60,63,63,61,61,61,63,65
11697,G. Gallon,27,France,ESTAC Troyes,GK,197,78.93,0,4,1600000,4000,2900000,0,84,96,237,217,101,41,347,1123,387,2,1,2,2,1,72,67,66,72,40,70,3,28,28,28,24,26,26,26,24,24,24,24,25,25,25,25,25,25,26,26,26,25,24,26,26,26,24,70
11698,J. Flores,22,Chile,CD Antofagasta,RM,164,64.86,0,8,1100000,2000,1700000,0,286,291,388,278,241,62,57,1603,337,3,3,3,2,1,76,60,60,72,26,43,6,62,62,62,67,65,65,65,67,65,65,65,66,58,58,58,66,49,44,44,44,49,45,35,35,35,45,67
11699,Anderson Silva,26,Brazil,Barnsley,CM,197,81.19,0,7,0,0,0,0,254,284,198,258,241,113,81,1429,400,3,1,0,0,3,72,63,70,72,57,66,3,68,68,68,71,68,68,68,71,72,72,72,71,71,71,71,71,66,68,68,68,66,64,60,60,60,64,68


In [8]:
fifa21['foot'].unique()

array([0, 1])

In [9]:
numerical = fifa21.select_dtypes(include=np.number)
display(numerical.dtypes)
display(numerical.shape)

age                           int64
height                        int64
weight_kg                   float64
foot                          int64
growth                        int64
value_€                       int64
wage_€                        int64
release_clause_€              int64
on_loan                       int64
attacking                     int64
skill                         int64
movement                      int64
power                         int64
mentality                     int64
defending                     int64
goalkeeping                   int64
total_stats                   int64
base_stats                    int64
weak_foot                     int64
skill_move                    int64
attacking_workrate            int64
defensive_workrate            int64
international_reputation      int64
pac                           int64
sho                           int64
pas                           int64
dri                           int64
def                         

(11683, 57)

In [23]:
categorical = fifa21.select_dtypes(include= object)
display(categorical.dtypes)
display(categorical.shape)

name             object
nationality      object
club             object
best_position    object
dtype: object

(11683, 4)

In [105]:
corr = numerical.corr(.loc[:,['ova']])

fig, ax = 

SyntaxError: invalid syntax (2013872706.py, line 1)

In [12]:
#X/y split

y = numerical['ovr']
X = numerical.drop(['ovr'], axis=1)
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)



In [15]:
#train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)


LinearRegression()

In [19]:
display(X_train.head())

,age,height,weight_kg,foot,growth,value_€,wage_€,release_clause_€,on_loan,attacking,skill,movement,power,mentality,defending,goalkeeping,total_stats,base_stats,weak_foot,skill_move,attacking_workrate,defensive_workrate,international_reputation,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
6412,23,197,77.11,0,5,925000,5000,1400000,0,229,204,316,287,250,199,43,1528,332,2,2,2,3,1,66,35,50,44,67,70,4,49,49,49,47,47,47,47,47,47,47,47,50,52,52,52,50,61,62,62,62,61,63,67,67,67,63
7471,20,197,73.03,0,13,140000,500,389000,0,196,168,255,253,193,153,50,1268,274,2,2,2,2,1,52,27,40,42,53,60,3,40,40,40,40,40,40,40,40,40,40,40,42,43,43,43,42,50,51,51,51,50,51,55,55,55,51
4272,30,197,77.11,0,0,80000,500,124000,0,231,217,186,271,228,172,59,1364,294,3,2,1,2,1,34,43,50,48,57,62,3,49,49,49,46,47,47,47,46,47,47,47,48,50,50,50,48,52,55,55,55,52,52,57,57,57,52
8151,24,164,69.85,1,7,900000,7000,2100000,0,287,305,327,297,269,178,45,1708,368,3,3,3,3,1,60,57,68,69,54,60,4,61,61,61,66,65,65,65,66,67,67,67,67,66,66,66,67,61,61,61,61,61,59,55,55,55,59
4610,23,164,68.95,0,7,1300000,9000,0,1,237,248,365,286,285,198,44,1663,366,3,2,2,2,1,81,32,49,66,65,73,6,53,53,53,58,55,55,55,58,55,55,55,61,55,55,55,61,66,61,61,61,66,67,66,66,66,67


In [20]:
display(y_train.head())

6412    67
7471    55
4272    57
8151    66
4610    69
Name: ovr, dtype: int64

In [21]:
#Metrics of R2 score, MSE, RMSE, MAE
predictions = lm.predict(X_train)
predictions_test = lm.predict(X_test)

print('Train R2 Score :',r2_score(y_train, predictions))
print('Test R2 Score :',r2_score(y_test, predictions_test))

Train R2 Score : 0.8739199678118718
Test R2 Score : 0.8713366696491642


In [22]:
mse = mean_squared_error(y_test, predictions_test)
rmse = math.sqrt(mse)
print('The RMSE is:',rmse)

The RMSE is: 2.442536243141305


In [ ]:
mae = 